### Tavily AI를 인터넷 검색 Tool로 설정

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

# API Key 환경변수 설정
load_dotenv()
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Tavily 검색 API를 Tool로 호출
search = TavilySearchResults()

### 벡터 DB 구축

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("data/★육아휴직제도 사용안내서_배포.pdf")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

### create_retriever_tool로 벡터 DB 검색 툴 설정

In [3]:
from langchain.tools.retriever import create_retriever_tool

# 벡터 DB 검색을 위한 검색 도구 설정
# 검색할 대상인 Retriever, 검색 Tool의 이름, Tool에 대한 설명을 인자로 설정
retriever_tool = create_retriever_tool(
    retriever,
    "parental_leave",
    "육아휴직과 관련한 정보를 검색합니다. 육아 휴직 관련한 질문이 입력되면 이 도구를 사용합니다.",
)

### Agent 구축

In [4]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

tools = [search, retriever_tool]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
# 벡터DB를 활용하는 질문
agent_executor.invoke({"input": "육아휴직은 어떻게 사용할 수 있어?"})



> Entering new AgentExecutor chain...

Invoking: `parental_leave` with `{'query': '육아휴직 사용 방법'}`


육아휴직중
일하는 엄마, 아빠를 위한 
육아휴직제도
사용안내서
육아휴직은 임신 중인 여성 근로자나, 
8세 또는 초등학교 2학년 이하 자녀를 가진 근로자라면 
누구나 사용할 수 있는 당연한 권리입니다.

일하는 엄마, 아빠를 위한 
육아휴직제도
사용안내서
육아휴직 등으로 불리한 처우를 받았다면 
전국 지방고용노동관서에서 운영 중인 
‘모성보호 신고센터’에 신고해주세요!

육아휴직제도는?
임신 중인 여성 근로자나, 8세 또는 초등학교 2학년 이하 자녀를 가진 근로자가 
자녀 양육을 위해 최대 1년간 육아휴직을 사용할 수 있는 제도입니다. 
육아휴직급여는?
육아휴직을 사용한 근로자에게 1년간 통상임금의 80%를 지원합니다.
 * 월 상한액 150만 원, 하한액 70만 원 
‘3+3 부모육아휴직제’도 있어요
생후 12개월 이내 자녀 돌봄을 위해 부모 모두 육아휴직을 
사용하는 경우 육아휴직급여 중 첫 3개월은 부모에게  
각각 통상임금의 100%를 지원합니다.
  1개월 상한액 200만 원 
 2개월 상한액 250만 원  
 3개월 상한액 300만 원  
통상임금이란?
근로자에게 정기적이고 일률적으로 소정근로 또는 총 근로에 대하여 지급하기로 정한 
금액을 의미합니다.(근로기준법 시행령 제6조제1항)
  통상임금은 초과근로수당 산정 등을 위한 기초임금으로서, 정기적·일률적·고정적
으로 지급되는 것인지를 기준으로 판단합니다.
 Tip

육아휴직 관련 자주 묻는 질문들
1   육아휴직을 신청해도 사업주가 허용해주지 않는데, 어떻게 하나요？
  육아휴직은 근로자의 권리이므로 요건을 갖춰 회사에 신청하면 
사업주는 반드시 허용해야 합니다.
   사업주가 육아휴직을 허용하지 않는 경우, 법률에 따라 500만 원 
이하의 벌금이 부과됩니다.(관련 신고절차는 Ⅳ~Ⅴ. 참조)
       * (예외) 

{'input': '육아휴직은 어떻게 사용할 수 있어?',
 'output': '육아휴직은 임신 중인 여성 근로자나 8세 이하 또는 초등학교 2학년 이하 자녀를 가진 근로자가 자녀 양육을 위해 사용할 수 있는 제도입니다. 육아휴직을 통해 최대 1년간 휴직할 수 있으며, 육아휴직급여로는 통상임금의 80%를 지원받습니다. 다만, 월급여는 상한액 150만 원, 하한액 70만 원으로 제한됩니다.\n\n또한, ‘3+3 부모육아휴직제’를 통해 부모가 모두 육아휴직을 사용할 경우, 첫 3개월 동안 각각 통상임금의 100%를 지원받을 수 있습니다. 이 경우 1개월 상한액은 200만 원, 2개월은 250만 원, 3개월은 300만 원입니다.\n\n육아휴직은 근로자의 권리이므로, 해당 요건을 갖춰 신청하면 사업주는 반드시 허용해야 하며, 이를 거부할 경우 법적으로 벌금이 부과될 수 있습니다. 만약 신청 후 사업주가 아무런 답변을 하지 않는 경우, 이는 묵시적으로 승인된 것으로 간주됩니다.\n\n육아휴직을 신청할 때는 관련 법률과 절차를 잘 이해하고, 필요한 경우 적절한 신고를 통해 권리를 보호할 수 있습니다.'}

In [7]:
# 인터넷 검색을 활용하는 질문
agent_executor.invoke({"input": "현재 대한민국의 인구는 몇명이야?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '현재 대한민국 인구'}`


[{'title': '대한민국의 인구 - 위키백과, 우리 모두의 백과사전', 'url': 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%EA%B5%AC', 'content': "목차\n\n대한민국의 인구\n\n대한민국의 인구는 아시아에서는 13번째, 세계에서는 29번째로 인구가 많은 나라이다. 1949년 정부수립 직후 2000만 명, 1967년에 3000만 명, 1983년에 4000만 명, 2012년 6월 23일[1]에 5000만 명을 넘었으며, 2020년부터 현재까지 인구가 감소하고 있다.\n\n인구피라미드 유형은 유소년인구가 감소하고, 노년인구가 증가함에 따라 인구 구조가 1950~70년대 '피라미드형', 1980~2010년대 '종형', 2020년대 이후로는 초저출산 심화에 따라 '항아리형' 내지 '역피라미드형'으로 변화하고 있다. [...] | 2020 통계청 인구주택총조사 연령 및 성별 인구\n총인구 | 남자 | 여자 | 성비 | 내국인 | 내국인-남자 | 내국인-여자 | 내국인-성비\n전체 | 51,829,136 | 25,915,207 | 25,913,929 | 100.0 | 50,133,493 | 24,972,588 | 25,160,905 | 99.3\n0~4세 | 1,722,081 | 883,196 | 838,885 | 105.3 | 1,684,917 | 864,276 | 820,641 | 105.3\n5~9세 | 2,264,595 | 1,161,247 | 1,103,348 | 105.2 | 2,238,916 | 1,148,248 | 1,090,668 | 105.3\n10~14세 | 2,267,481 | 1,168,937 | 1,098,544

{'input': '현재 대한민국의 인구는 몇명이야?',
 'output': '현재 대한민국의 총 인구는 약 51,684,564명입니다. 인구는 2020년부터 감소세를 보이고 있으며, 노인 인구가 증가하고 유소년 인구가 감소하는 구조적 변화가 나타나고 있습니다. \n\n더 자세한 정보는 [위키백과](https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%EA%B5%AC)와 [나무위키](https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/%EC%9D%B8%EA%B5%AC)에서 확인하실 수 있습니다.'}